In [1]:
import importlib
import src.utils
import src.models
import src.counterfactual

importlib.reload(src.utils)
importlib.reload(src.models)
importlib.reload(src.counterfactual)

from src.utils import load_data, load_model, DatasetMetadata, clean_instance
from src.counterfactual import newton_op, distance
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sympy as sp
# str to sympy
from sympy.parsing.sympy_parser import parse_expr

from torch.utils.data import DataLoader
from src.models import LogisticModel
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = device if not torch.backends.mps.is_available() else torch.device("mps")


In [2]:
class State:
    def __init__(self, model, metadata, max_epochs, dx_scaled, mean_scaled, upd_weights):
        self.model: LogisticModel = model
        self.metadata: DatasetMetadata = metadata
        self.dx_scaled: torch.Tensor = dx_scaled
        self.mean_scaled: torch.Tensor = mean_scaled
        self.epochs: int = 0
        self.max_epochs: int = max_epochs
        self.upd_weights: torch.Tensor = upd_weights # columns to be updated
        self.apply_reg = False # When to apply integer regularization
        self.reg_vars = False # When to apply nº variables regularization


In [3]:
def unscale_instance(instance: torch.Tensor, metadata: DatasetMetadata, inplace: bool = False):
    cols_to_unscale = instance[metadata.cols_for_scaler].reshape(1, -1)
    mean = torch.tensor(metadata.scaler.mean_)
    std = torch.tensor(metadata.scaler.scale_)
    unscaled_cols = cols_to_unscale * std + mean
    if inplace:
        instance[metadata.cols_for_scaler] = torch.tensor(unscaled_cols, dtype=torch.float32).to(device)
        return instance
    else:
        instance_clone = instance.clone()
        instance_clone[metadata.cols_for_scaler] = torch.tensor(unscaled_cols, dtype=torch.float32).to(device)
        return instance_clone
    
def scale_instance(instance: torch.Tensor, metadata: DatasetMetadata, inplace: bool = False):
    cols_to_scale = instance[metadata.cols_for_scaler].reshape(1, -1)
    mean = torch.tensor(metadata.scaler.mean_)
    std = torch.tensor(metadata.scaler.scale_)
    scaled_cols = (cols_to_scale - mean) / std
    if inplace:
        instance[metadata.cols_for_scaler] = torch.tensor(scaled_cols, dtype=torch.float32).to(device)
        return instance
    else:
        instance_clone = instance.clone()
        instance_clone[metadata.cols_for_scaler] = torch.tensor(scaled_cols, dtype=torch.float32).to(device)
        return instance_clone
    
def round_instance(instance: torch.Tensor, metadata: DatasetMetadata):
    unscaled_person = unscale_instance(instance, metadata)
    unscaled_person[metadata.int_cols == 1] = torch.round(unscaled_person[metadata.int_cols == 1])
    person_new = scale_instance(unscaled_person, metadata)
    return person_new


def unscale_batch(batch: torch.Tensor, metadata: DatasetMetadata, inplace: bool = False):
    cols_to_unscale = torch.tensor(batch[:, metadata.cols_for_scaler], dtype=torch.float32)
    mean = torch.tensor(metadata.scaler.mean_, dtype=torch.float32)
    std = torch.tensor(metadata.scaler.scale_, dtype=torch.float32)
    unscaled_cols = cols_to_unscale * std + mean
    if inplace:
        batch[:, metadata.cols_for_scaler] = torch.tensor(unscaled_cols, dtype=torch.float32).to(device)
        return batch
    else:
        batch_clone = batch.clone()
        batch_clone[:, metadata.cols_for_scaler] = torch.tensor(unscaled_cols, dtype=torch.float32).to(device)
        return batch_clone
    
def scale_batch(batch: torch.Tensor, metadata: DatasetMetadata, inplace: bool = False):
    cols_to_scale = torch.tensor(batch[:, metadata.cols_for_scaler], dtype=torch.float32)
    mean = torch.tensor(metadata.scaler.mean_, dtype=torch.float32)
    std = torch.tensor(metadata.scaler.scale_, dtype=torch.float32)
    scaled_cols = (cols_to_scale - mean) / std
    if inplace:
        batch[:, metadata.cols_for_scaler] = torch.tensor(scaled_cols, dtype=torch.float32).to(device)
        return batch
    else:
        batch_clone = batch.clone()
        batch_clone[:, metadata.cols_for_scaler] = torch.tensor(scaled_cols, dtype=torch.float32).to(device)
        return batch_clone
    
def round_batch(batch: torch.Tensor, metadata: DatasetMetadata):
    unscaled_person = unscale_batch(batch, metadata)
    unscaled_person[metadata.int_cols == 1] = torch.round(unscaled_person[metadata.int_cols == 1])
    person_new = scale_batch(unscaled_person, metadata)
    return person_new

In [4]:
filename = 'data/Loan_default.csv'
model_name = "model_small"
model_dict = "models/"+model_name+".pth"

In [5]:
# load the model
test_data: DataLoader
_, _, test_data, _, metadata = load_data(filename, batch_size=1024)

inputs = next(iter(test_data))[0].to(torch.float32).to(device)

# define model
model = load_model(model_name).to(torch.float32).to(device)

torch.save(model.state_dict(), model_dict)


## Extract model equation

In [6]:
# import sympy as sp
# import torch

# def extract_symbolic_equation(model: torch.nn.Module, instance: torch.Tensor):
#     """
#     Extracts a symbolic equation from a trained PyTorch model.
#     Assumes a feedforward structure with linear layers and activations.
#     """
#     # Define symbolic variables for input features
#     x2, x3 = sp.symbols('x2 x3')  # Inputs
#     # constants = sp.symbols(f'c1:{model.input_dim + 1}')  # Constants for other features
    
#     # Build input vector with constants
#     x = [instance[i].item() if i not in [1, 2] else (x2 if i == 1 else x3) for i in range(model.input_dim)]
    
#     # Convert to a sympy matrix
#     X = sp.Matrix(x)
#     activations = []

#     # Iterate over layers
#     for layer in model.layers:
#         if isinstance(layer, torch.nn.Linear):
#             W = sp.Matrix(layer.weight.detach().numpy())  # Extract weight matrix
#             b = sp.Matrix(layer.bias.detach().numpy())    # Extract bias
#             X = W * X + b  # Apply linear transformation
#         elif isinstance(layer, torch.nn.ReLU):
#             activations.append(X)
#             X = X.applyfunc(lambda val: sp.Max(0, val))  # ReLU activation
#         elif isinstance(layer, torch.nn.Sigmoid):
#             activations.append(X)
#             X = X.applyfunc(lambda val: 1 / (1 + sp.exp(-val)))  # Sigmoid activation
#         # X.subs({sp.symbols(f'c{i+1}'): val for i, val in enumerate(inputs[0]) if i != 1 and i != 2})
#         print("Done: ", layer)
#         # print(X)


#     # Apply softmax at the end
#     denominator = sp.Add(*(sp.exp(e) for e in X))
#     softmax_expr = sp.Matrix([sp.exp(e) / denominator for e in X])

#     return softmax_expr, activations # .simplify()

# # Example usage
# model_sym = LogisticModel(inputs.shape[1], hidden_sizes=[16, 8])
# model_sym.load_state_dict(torch.load(model_dict))  # Load trained weights
# symbolic_eq, activations = extract_symbolic_equation(model_sym, inputs[0])
# model_eq = symbolic_eq[0]
# print(symbolic_eq)


In [7]:
# # with open('small.txt', 'w') as f:
# #     f.write(str(symbolic_eq))
# with open('small.txt', 'r') as f:
#     symbolic_eq1 = f.read()
#     symbolic_eq1 = parse_expr(symbolic_eq1)

## Training

In [8]:

person: torch.Tensor = inputs[0].to(torch.float32).to(device)
outputs = model(inputs).argmax(dim=1)
inputs_useful = inputs[outputs == 1]
# metadata.cols_for_mask = [False] * len(metadata.cols_for_mask)
# metadata.cols_for_mask[1] = True
# metadata.cols_for_mask[2] = True
# metadata.cols_for_mask[3] = True
# metadata.cols_for_mask[4] = True
# metadata.cols_for_mask[5] = True
# metadata.cols_for_mask[6] = True
# metadata.cols_for_mask[7] = True
# metadata.cols_for_mask[8] = True

weights = torch.tensor(metadata.cols_for_mask, dtype=torch.float32).to(device)


In [9]:
person = inputs_useful[0].to(torch.float32).to(device)
# weights = torch.tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int).to(device)

In [10]:
class Checks:
    def __init__(self, model: LogisticModel, metadata: DatasetMetadata, reg_int: bool = False, reg_clamp: bool = False, noise: float = 1e-3, n_points: int = 10000, noise_int: float = 2.5):
        self.model = model
        self.metadata = metadata
        self.reg_int = reg_int
        self.reg_clamp = reg_clamp
        self.noise = noise
        self.n_points = n_points
        self.noise_int = noise_int
        self.distance_threshold = 1e-4
        self.sorted_points = None
        self.model.eval()


    def __call__(self, person: torch.Tensor, person_new: torch.Tensor, weights: torch.Tensor):
        """
        Check if the new person is a valid counterfactual.
        """
        # Check if the new person is a valid counterfactual
        if not self.validity_check(person, person_new):
            print("The new person is not a valid counterfactual.")
            return False
        
        # Check if the new person is plausible
        if self.reg_clamp and not self.plausibility_check(person_new):
            print("The new person is not plausible.")
            return False

        # Check if the new person is minimal
        if self.reg_int:
            valid, sorted_points = self.integer_minimality_check(person, person_new, weights)
            if not valid:
                print("The new person is not integer minimal.", "The length of the sorted points is: ", len(sorted_points))
                self.sorted_points = sorted_points
                return False

        else:
            valid, sorted_points = self.minimality_check(person, person_new, weights)
            if not valid:
                print("The new person is not minimal.", "The length of the sorted points is: ", len(sorted_points))
                self.sorted_points = sorted_points
                return False
            
        return True


    def minimality_check(
        self,
        person: torch.Tensor,
        person_new: torch.Tensor,
        weights: torch.Tensor,
        ):
        points = torch.tensor(
            np.random.uniform(-self.noise, self.noise, (self.n_points, person_new.shape[0]))
            * weights.numpy()
            + person_new.detach().cpu().numpy().reshape(-1),
            dtype=torch.float32,
        ).to(device)
        points = (
            torch.clamp(points, self.metadata.min_values, self.metadata.max_values)
            if self.reg_clamp
            else points
        )
        outputs = model(points)
        # pandas dataset
        b = pd.DataFrame(points, columns=self.metadata.columns)
        b["output"] = torch.argmax(outputs, dim=1).detach().cpu().numpy()
        distances = torch.tensor([distance(person, p, weights) for p in points])
        b["distance"] = distances.detach().cpu().numpy()

        d = distance(person, person_new, weights).item()
        sorted_b = b[b["distance"] <= d][b["output"] == 0].sort_values(by="distance")
        # sorted_b = sorted_b[(sorted_b["distance"] - d) < self.distance_threshold]
        return len(sorted_b) == 0, sorted_b

    def integer_minimality_check(
        self,
        person: torch.Tensor,
        person_new_int: torch.Tensor,
        weights: torch.Tensor,
    ):
        w = ((weights != 0) & ~self.metadata.int_cols) * weights
        noise_tensor = np.random.uniform(
            -self.noise, self.noise, (self.n_points, person_new_int.shape[0])
        ) * w.numpy()
        rounded_noise_tensor = np.random.randint(
            -self.noise_int, self.noise_int, (self.n_points, person_new_int.shape[0])
        ) * (((weights != 0) & self.metadata.int_cols) * weights).numpy()

        points = scale_batch(
            torch.tensor(
                noise_tensor
                + rounded_noise_tensor
                + unscale_instance(person_new_int, self.metadata).detach().cpu().numpy().reshape(-1),
                dtype=torch.float32,
            ),
            self.metadata,
        ).to(device)
        
        points = (
            torch.clamp(points, self.metadata.min_values, self.metadata.max_values)
            if self.reg_clamp
            else points
        )

        outputs = model(points)

        points_unscaled = unscale_batch(points, self.metadata)
        b = pd.DataFrame(points_unscaled, columns=self.metadata.columns)

        # add person_new_int to the dataframe
        b["output"] = torch.argmax(outputs, dim=1).detach().cpu().numpy()

        distances = torch.tensor([distance(person, p, weights) for p in points])
        b["distance"] = distances.detach().cpu().numpy()

        d = distance(person, person_new_int, ((weights != 0) & ~self.metadata.int_cols) * weights).item()
        sorted_b = b[b["distance"] < d][b["output"] == 0].sort_values(by="distance")

        return len(sorted_b) == 0, sorted_b
    
    def validity_check(
      self,
      person: torch.Tensor,
      person_new: torch.Tensor,
    ):
        return (
            (self.model(person_new.unsqueeze(0))[0][metadata.good_class].item() >= 0.5)
            != (self.model(person.unsqueeze(0))[0][metadata.good_class].item() >= 0.5)
        )
    
    def plausibility_check(
        self,
        person_new: torch.Tensor,
    ):
        # Check if the new person is plausible
        return (
            torch.clamp(
                person_new,
                self.metadata.min_values,
                self.metadata.max_values,
            )
            == person_new
        ).all().item()
            

## Global check

In [11]:
# import src.counterfactual

# importlib.reload(src.counterfactual)
# from src.counterfactual import newton_op, distance
# person = inputs_useful[0]
# p_new, state_p = newton_op(model, person, metadata, weights, 0.1, reg_int=False, reg_vars=False, reg_clamp=True, print_=False)
# torch.manual_seed(torch.randint(0, 100, (1,)).item())
# n = 5
# num_points = 10000
# num_linspace = 5000
# indexes = torch.nonzero(weights).reshape(-1)
# print(indexes)
# sampled_indexes = torch.tensor([6, 8]) # indexes #[torch.randint(0, len(indexes), (n,))]
# print(sampled_indexes)

# for sample_var in sampled_indexes:
#     w = weights.clone()
#     w[sample_var] = 0
#     print("Sampled variable:", sample_var.item())
#     x = p_new.repeat(num_points*num_linspace, 1)
#     print("points repeated")
#     # print("x:", x[:, w != 0])

#     # print(x)
#     x[:, w != 0] = (torch.distributions.uniform.Uniform(metadata.min_values, metadata.max_values).sample((num_points,)) * w)[:, w != 0].repeat(num_linspace, 1)
#     print("points generated")

#     x[:, sample_var] = torch.linspace(metadata.min_values[sample_var], metadata.max_values[sample_var], num_linspace).repeat(num_points)
#     print("linspace generated")

#     x = x[model(x)[:, 0] > metadata.threshold]
#     print("model filtered")

#     # calculate the distance
#     dists = distance(person, x, weights, state_p, with_sum=False)
#     print(torch.min(dists), distance(person, p_new, weights, state_p))
#     x = x[dists < distance(person, p_new, weights, state_p)]
#     print("distance filtered")
#     print(len(x))

In [12]:
import src.counterfactual

importlib.reload(src.counterfactual)
from src.counterfactual import newton_op

reg_int=False
reg_clamp=False

metadata.threshold = 0.5 + 1e-7
person = inputs_useful[29]
p_new, state_p = newton_op(model, person, metadata, weights, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, print_=True, der = True)
p_new_1, state_p = newton_op(model, person, metadata, weights, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, print_=True, der = False)

display(pd.DataFrame([unscale_instance(person, metadata).detach().numpy(), unscale_instance(p_new, metadata).detach().numpy(), unscale_instance(p_new_1, metadata).detach().numpy()], columns=metadata.columns))

Epoch: 0
Using gradient descent: 0.0064483825117349625
Changes:  delta1: -0.05039399862289429  delta_l: 0.00012806033191736788
dist: 0.011982154101133347 , threshold: 0.3815811574459076
Epoch: 1
Using gradient descent: 0.007333584129810333
Changes:  delta1: 0.3445945084095001  delta_l: -0.005485714878886938
dist: 0.4082350730895996 , threshold: 0.38807809352874756
Epoch: 2
Using gradient descent: 0.0031484123319387436
Changes:  delta1: -2.375239610671997  delta_l: 0.01428130827844143
dist: 20.423843383789062 , threshold: -0.48134511709213257
Epoch: 3
Using gradient descent: 0.008992389775812626
Changes:  delta1: 16.85065269470215  delta_l: -0.2837565541267395
dist: 1028.82373046875 , threshold: 0.3935144543647766
Epoch: 4
Using gradient descent: 1.329569829613183e-15
Changes:  delta1: -118.15690612792969  delta_l: 3.2328369039928695e-13
dist: 50412.36328125 , threshold: -0.4897453188896179
Epoch: 5
Using gradient descent: 0.0
Changes:  delta1: 827.0983276367188  delta_l: 0.0
dist: 2470

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education_High School,...,EmploymentType_Unemployed,MaritalStatus_Married,MaritalStatus_Single,HasMortgage_Yes,HasDependents_Yes,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_Yes
0,20.0,1.568100e+04,2.227720e+05,5.990000e+02,9.999999e+00,2.000000e+00,2.142000e+01,2.400000e+01,5.200000e-01,1.0,...,0.0,1.0,0.0,0.000000e+00,1.000000e+00,1.0,0.0,0.0,0.0,0.0
1,20.0,1.574983e+25,-2.432196e+25,2.172996e+22,1.313965e+22,-1.339862e+20,-3.306900e+21,-3.646444e+20,-1.612337e+19,1.0,...,0.0,1.0,0.0,1.342396e+20,2.018959e+20,1.0,0.0,0.0,0.0,0.0
2,20.0,6.347383e+04,1.490316e+05,6.638987e+02,5.023692e+01,1.615228e+00,1.105524e+01,2.321143e+01,4.692102e-01,1.0,...,0.0,1.0,0.0,4.275824e-01,1.613099e+00,1.0,0.0,0.0,0.0,0.0


## Trials

### Only 1 person

In [13]:
import src.counterfactual

importlib.reload(src.counterfactual)
from src.counterfactual import newton_op

reg_int=False
reg_clamp=True

metadata.threshold = 0.5 + 1e-7
person = inputs_useful[24]
p_new, state_p = newton_op(model, person, metadata, weights, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, print_=True)
check = Checks(model, metadata, reg_int=reg_int, reg_clamp=reg_clamp)
valid = check(person, p_new, weights)
print("Valid:", valid)
display(pd.DataFrame([unscale_instance(p_new, metadata).detach().numpy()], columns=metadata.columns))
display(check.sorted_points)
print(model(p_new.unsqueeze(0))[0][metadata.good_class].item(), model(p_new.unsqueeze(0)).argmax(dim=1))
if check.sorted_points is not None:
    minimal = torch.tensor(check.sorted_points.iloc[0, :-2].values).float()
    print(distance(person, p_new, weights).item(), distance(person, minimal, weights).item())
    display(pd.DataFrame(unscale_batch(torch.tensor(check.sorted_points.to_numpy()[:, :-2]).float(), metadata), columns=metadata.columns))

Epoch: 0
Changes:  delta1: -0.11136412620544434  delta_l: -0.4209531843662262
dist: 0.059967249631881714 , threshold: -0.01748371124267578
Epoch: 1
Changes:  delta1: 0.0135987913236022  delta_l: 0.1636035442352295
dist: 0.04620802775025368 , threshold: -0.00017887353897094727
Epoch: 2
Changes:  delta1: 0.00014118890976533294  delta_l: 0.0030081146396696568
dist: 0.0429217629134655 , threshold: 0.008349418640136719
Epoch: 3
Changes:  delta1: -0.0071990895085036755  delta_l: -0.055533722043037415
dist: 0.049494508653879166 , threshold: -4.971027374267578e-05
Epoch: 4
Changes:  delta1: 4.2436455260030925e-05  delta_l: 0.0006785510340705514
dist: 0.049454476684331894 , threshold: 0.0
Epoch: 5
Changes:  delta1: -5.5600071213746105e-09  delta_l: -4.28382591621812e-08
dist: 0.049454476684331894 , threshold: 0.0
Original output: tensor([[0.3895, 0.6105]], grad_fn=<DifferentiableGraphBackward>)
New output: tensor([[0.5000, 0.5000]], grad_fn=<DifferentiableGraphBackward>)
Regularization strength

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education_High School,...,EmploymentType_Unemployed,MaritalStatus_Married,MaritalStatus_Single,HasMortgage_Yes,HasDependents_Yes,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_Yes
0,29.0,72707.5625,151759.875,349.318878,16.398561,1.0,4.967973,35.939743,0.235276,1.0,...,1.0,0.0,1.0,0.035825,1.0,0.0,0.0,0.0,1.0,0.0


None

0.5000001192092896 tensor([0])


### 1 person, different weights


In [14]:
import src.counterfactual

importlib.reload(src.counterfactual)
from src.counterfactual import newton_op

reg_int=True
reg_clamp=False
check = Checks(model, metadata, reg_int=reg_int, reg_clamp=reg_clamp)

person = inputs_useful[1]

weights1 = weights.clone()
print(weights1)
p_new1, state_p = newton_op(model, person, metadata, weights1, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, print_=True)
valid1 = check(person, p_new, weights)
print("Valid1:", valid1)

weights2 = weights.clone()
weights2[1] = 2
print(weights2)
p_new2, state_p = newton_op(model, person, metadata, weights2, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, print_=True)
valid2 = check(person, p_new, weights)
print("Valid2:", valid2)

display(pd.DataFrame([unscale_instance(person, metadata).detach().numpy(), unscale_instance(p_new1, metadata).detach().numpy(), unscale_instance(p_new2, metadata).detach().numpy()], columns=metadata.columns))
# display(check.sorted_points)
# print(model(p_new.unsqueeze(0))[0][metadata.good_class].item(), model(p_new.unsqueeze(0)).argmax(dim=1))
# if check.sorted_points is not None:
#     minimal = torch.tensor(check.sorted_points.iloc[0, :-2].values).float()
#     print(distance(person, p_new, weights).item(), distance(person, minimal, weights).item())
#     display(pd.DataFrame(unscale_batch(torch.tensor(check.sorted_points.to_numpy()[:, :-2]).float(), metadata), columns=metadata.columns))

tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0.])
Epoch: 0
ONLY MODEL DERIVATIVE: 0.1418013870716095
Changes:  delta1: -0.1343325674533844  delta_l: 0.0
dist: 0.0886342003941536 , threshold: 0.24221941828727722
Epoch: 1
ONLY MODEL DERIVATIVE: 0.23989199101924896
Changes:  delta1: -0.10931722074747086  delta_l: 0.0
dist: 0.2915289103984833 , threshold: 0.16995853185653687
Epoch: 2
Changes:  delta1: -0.21144315600395203  delta_l: -4.669877052307129
dist: 1.014951467514038 , threshold: -0.0643581748008728
Epoch: 3
Changes:  delta1: -2.1112023205205332e-06  delta_l: 2.4245948791503906
dist: 77292.0625 , threshold: -0.0015573501586914062
Epoch: 1
Changes:  delta1: -2.3548711851617554e-06  delta_l: -0.005281630903482437
dist: 77290.96875 , threshold: -2.5033950805664062e-06
Epoch: 2
Changes:  delta1: -1.5838990066185943e-06  delta_l: -0.0037707462906837463
dist: 77290.5625 , threshold: -5.960464477539063e-08
Epoch: 3
Changes:  de

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education_High School,...,EmploymentType_Unemployed,MaritalStatus_Married,MaritalStatus_Single,HasMortgage_Yes,HasDependents_Yes,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_Yes
0,19.0,29467.0,151769.0,606.0,33.0,1.0,6.630000,24.0,0.480000,1.0,...,1.0,0.0,1.0,1.000000,0.000000,1.0,0.0,0.0,0.0,0.0
1,19.0,47199.0,124394.0,630.0,48.0,1.0,3.558925,24.0,0.465305,1.0,...,1.0,0.0,1.0,1.129511,0.177241,1.0,0.0,0.0,0.0,0.0
2,19.0,39338.0,121286.0,633.0,50.0,1.0,3.131066,24.0,0.463308,1.0,...,1.0,0.0,1.0,1.147600,0.201816,1.0,0.0,0.0,0.0,0.0


### 1 person, different flags (clamp, int)

In [15]:
import src.counterfactual

importlib.reload(src.counterfactual)
from src.counterfactual import newton_op, distance, unscale_instance, scale_instance
person = inputs_useful[47]
person_new, state = newton_op(model, person, metadata, weights, 0.1, print_=True)
person_new_clamp, _ = newton_op(model, person, metadata, weights, 0.1, reg_clamp=True, print_=True)
person_new_int, _ = newton_op(model, person, metadata, weights, reg_int=True, print_=True)
person_new_clamp_int, _ = newton_op(model, person, metadata, weights, 0.1, reg_int=True, reg_clamp=True, print_=True)
# person_new_vars, _ = newton_op(model, person, metadata, weights, reg_vars=True, print_=True)
# person_new_int_vars, _ = newton_op(model, person, metadata, weights, 0.1, reg_int=True, reg_vars=True, print_=True)

names = ['person', 'person_new', 'person_new_clamp','person_new_int', 'person_new_clamp_int']
ps = [eval(i) for i in names]
outputs = [model(p.unsqueeze(0))[0][metadata.good_class].item() for p in ps]

distances = [distance(person, p, weights, state=state).item() for p in ps]

a = pd.DataFrame([unscale_instance(x, metadata).detach().cpu().numpy().reshape(-1) for x in ps], columns=metadata.columns)
a['output'] = outputs
a['distance'] = distances
# set index
a['names'] = names
a = a.set_index('names')
print(a.columns)
a

Epoch: 0
Changes:  delta1: -0.034573525190353394  delta_l: 0.24713248014450073
dist: 0.0057977959513664246 , threshold: -0.001454770565032959
Epoch: 1
Changes:  delta1: 0.0011711370898410678  delta_l: -0.010803382843732834
dist: 0.005413780454546213 , threshold: -1.430511474609375e-06
Epoch: 2
Changes:  delta1: 1.1767133401008323e-06  delta_l: -1.5147255908232182e-05
dist: 0.005413417238742113 , threshold: -5.960464477539063e-08
Epoch: 3
Changes:  delta1: 8.933087158879971e-09  delta_l: 1.645505705027972e-07
dist: 0.005413396749645472 , threshold: 0.0
Original output: tensor([[0.4596, 0.5404]], grad_fn=<DifferentiableGraphBackward>)
New output: tensor([[0.5000, 0.5000]], grad_fn=<DifferentiableGraphBackward>)
Regularization strength: 0.2599424421787262
Epochs: 4
Epoch: 0
Changes:  delta1: -0.034573525190353394  delta_l: 0.24713248014450073
dist: 0.0057977959513664246 , threshold: -0.001454770565032959
Epoch: 1
Changes:  delta1: 0.0011711370898410678  delta_l: -0.010803382843732834
dist

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education_High School,...,MaritalStatus_Single,HasMortgage_Yes,HasDependents_Yes,LoanPurpose_Business,LoanPurpose_Education,LoanPurpose_Home,LoanPurpose_Other,HasCoSigner_Yes,output,distance
names,,,,,,,,,,,,,,,,,,,,,
person,54.0,34432.000000,209972.000000,820.000000,92.000000,4.000000,21.840000,60.000000,0.890000,1.0,...,0.0,1.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.459618,0.000000
person_new,54.0,35733.410156,207960.265625,821.785034,93.111809,3.989704,21.558788,59.981865,0.888719,1.0,...,0.0,1.012439,0.016371,0.0,0.0,0.0,1.0,0.0,0.500000,0.005413
person_new_clamp,54.0,35771.710938,207901.046875,821.837402,93.144508,3.989401,21.550522,59.981335,0.888681,1.0,...,0.0,1.000000,0.016854,0.0,0.0,0.0,1.0,0.0,0.500000,0.005573
person_new_int,54.0,35733.003906,207960.000000,822.000000,93.000000,4.000000,21.536942,60.000000,0.888620,1.0,...,0.0,1.013405,0.017642,0.0,0.0,0.0,1.0,0.0,0.500000,0.005525
person_new_clamp_int,54.0,35733.003906,207960.000000,822.000000,93.000000,4.000000,21.514563,60.000000,0.888517,1.0,...,0.0,1.000000,0.018947,0.0,0.0,0.0,1.0,0.0,0.500000,0.005718


### Batch

In [16]:
import src.counterfactual
importlib.reload(src.counterfactual)
from src.counterfactual import newton_op, distance, minimality_check, integer_minimality_check

reg_int = False
reg_clamp = True
metadata.threshold = 0.5 + 1e-7

successes = 0
epochs = 0
bad_idxs = []
total = 0
check = Checks(model, metadata, reg_int=reg_int, reg_clamp=reg_clamp)
for idx, p in enumerate(inputs_useful):
    print("Person:", idx)
    p_new, ep = newton_op(model, p, metadata, weights, 0.2, reg_int=reg_int, reg_clamp=reg_clamp, der=True)
    # TODO: poner la minimalidad
    valid = check(p, p_new, weights)
    successes += valid # and (((state_p.metadata.max_values < p_new) | (state_p.metadata.min_values > p_new)).sum() == 0))
    # print("Person:", idx, "Rate of grad desc:",minimality_check(p, p_new, weights, ep, model))
    epochs += ep.epochs
    total += 1
    if not valid:
        bad_idxs.append(idx)
        # print(idx, valid)
print("Successes:", successes, "Total:", total)
print("Average epochs:", epochs / total)
print("Success rate:", successes / total)

Person: 0
The new person is not plausible.
Person: 1
Person: 2
The new person is not plausible.
Person: 3
Person: 4
The new person is not plausible.
Person: 5
Person: 6
Person: 7
The new person is not plausible.
Person: 8
Person: 9
Person: 10
The new person is not plausible.
Person: 11
Person: 12
Person: 13
Person: 14
Person: 15
Person: 16
Person: 17
Person: 18
The new person is not plausible.
Person: 19
Person: 20
Person: 21
Person: 22
The new person is not plausible.
Person: 23
The new person is not plausible.
Person: 24
Person: 25
The new person is not plausible.
Person: 26
The new person is not plausible.
Person: 27
Person: 28
The new person is not plausible.
Person: 29
The new person is not plausible.
Person: 30
Person: 31
Person: 32
The new person is not plausible.
Person: 33
The new person is not plausible.
Person: 34
Person: 35
Person: 36
Person: 37
Person: 38
The new person is not plausible.
Person: 39
Person: 40
Person: 41
The new person is not plausible.
Person: 42
Person: 4

### All batches

In [17]:
# successes = 0
# bad_idxs = []
# total = 0
# for i, inputs in enumerate(test_data):
#     print(i, end='\r')
#     outputs = model(inputs[0]).argmax(dim=1)
#     inputs_useful = inputs[0][outputs == 1]
#     for idx, p in enumerate(inputs_useful):
#         _, ep = newton_op(model, p, weights, 0.1) #if idx not in [103, 105, 237, 406, 417, 450] else None
#         # print("Person:", idx, "Success:", not ep)
#         successes += ep
#         total += 1
#         # if not ep:
#         #     bad_idxs.append(idx)
#     print(successes/total)
# print("Successes:", successes, "Total:", total)
# print("Success rate:", successes / total)